In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [2]:
# model = "AdaptableLSTM"
model = "LogisticRegressor"
learning_rate = .03
epochs = 190
seed = 1
include_state = True
estate = True
fullq = False
respond_perc = .50

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    numValFolds = 5,
    data_kw={"minw": 2,
            "maxw": 29,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 200,
        "lr_step_mult": .9, 
        "lr_step_epochs": 60,
        "opt_kw": {
            "lr": learning_rate,
        }
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })



report = e.run()




268 537
(6904, 56) (6904, 8)
0	 [0.10722040385007858]
5	 [0.09395033121109009]
10	 [0.07524826377630234]
15	 [0.06293371319770813]
20	 [0.06052512675523758]
25	 [0.0564492866396904]
30	 [0.05491332337260246]
35	 [0.05267776921391487]
40	 [0.0514175221323967]
45	 [0.050859320908784866]
50	 [0.05044882372021675]
55	 [0.049964506179094315]
60	 [0.04975331947207451]
65	 [0.049603112041950226]
70	 [0.04947585612535477]
75	 [0.04937669634819031]
80	 [0.049284011125564575]
85	 [0.04920724034309387]
90	 [0.04914553463459015]
95	 [0.04909445717930794]
100	 [0.04904720559716225]
105	 [0.04900584742426872]
110	 [0.04897022992372513]
115	 [0.048938605934381485]
120	 [0.048910316079854965]
125	 [0.048887621611356735]
130	 [0.04886728897690773]
135	 [0.04884875565767288]
140	 [0.04883197322487831]
145	 [0.04881671816110611]
150	 [0.04880277067422867]
155	 [0.04879003018140793]
160	 [0.04877835884690285]
165	 [0.04876764118671417]
170	 [0.04875778406858444]
175	 [0.04874871298670769]
180	 [0.04874033